In [30]:
import tensorflow as tf
import matplotlib.pyplot as plt
import glob, os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers
from tensorflow.python.data.ops import dataset_ops
from tensorflow.python.ops import image_ops
from tensorflow.python.ops import io_ops
from tensorflow.python.util.tf_export import keras_export

In [36]:
import skimage

ImportError: No module named 'skimage'

In [31]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

ImportError: No module named 'tensorflow_datasets'

In [32]:
img_height = 300
img_width = 300
batch_size = 5

In [34]:
ds_train = tf.image(
    'Alzheimer_s Dataset/train',
    labels="inferred",
    label_mode = 'int',
    color_mode='greyscale',
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle = True,
    seed=123,
    validation_split=0.1,
    subset='training',)

ds_validation = tf.keras.preprocessing.image(
    'Alzheimer_s Dataset/train',
    labels="inferred",
    label_mode = 'int',
    color_mode='greyscale',
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle = True,
    seed=123,
    validation_split=0.1,
    subset='validation',)

TypeError: 'module' object is not callable